In [1]:
require(tidyverse)
require(data.table)

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [121]:
cohort_size <- 337151

### compute expected genotype counts under HWE

In [48]:
annot.tbl <- '/oak/stanford/groups/mrivas/private_data/ukbb/variant_filtering/variant_filter_table.tsv.gz'


In [49]:
annot.arr <- fread(
    cmd=paste0('zcat ', annot.tbl),
    sep='\t', data.table=FALSE
) %>% mutate(
    MAF=pmin(freq, 1-freq)
) %>%
mutate(
    variant = paste(CHROM, POS, REF, ALT, sep=':'),
    is_outside_of_MHC = (as.numeric(CHROM) == 6 & as.numeric(POS) < 25477797) | ( as.numeric(CHROM) == 6 & 36448354 < as.numeric(POS)) | as.numeric(CHROM) != 6
)


In [135]:
expected_genotype_freq <- annot.arr %>% 
filter(ID %in% c('rs200058074', 'rs28991002', 'rs28991009', 'rs143435072')) %>%
select(ID, REF, ALT, freq) %>%
mutate(
    REF_REF = (1 - freq) * (1 - freq),
    REF_ALT = 2 * freq * (1 - freq),
    ALT_ALT = freq * freq
) %>%
gather(
  genotype_s, expected, -ID, -REF, -ALT, -freq
) %>%
arrange(ID) %>%
mutate(
    g_REF_REF = paste(REF, REF, sep='/'),
    g_REF_ALT = paste(REF, ALT, sep='/'),
    g_ALT_ALT = paste(ALT, ALT, sep='/'),
    genotype = if_else(genotype_s == 'REF_REF', g_REF_REF, if_else(genotype_s == 'REF_ALT', g_REF_ALT, g_ALT_ALT))
) %>%
select(-g_REF_REF, -g_REF_ALT, -g_ALT_ALT, -genotype_s, -freq) %>%
select(ID, genotype, expected) %>%
arrange(ID, -expected) %>%
bind_rows(
    raw_df %>% gather(variant_REF, genotype, -IID) %>% 
    separate(variant_REF, c("ID", "REF"), sep = "_") %>%
    count(ID, genotype) %>%
    filter(is.na(genotype)) %>%
    mutate(expected = n / cohort_size) %>% 
    select(-n)    
)



### read genotype counts (raw data)

In [2]:
raw_f <- 'ukb24983_cal_cALL_v2_hg19_ANGPTL7_protein-altering_vars.raw'


In [7]:
raw_df <- fread(
    cmd=paste0('cat ', raw_f, ' | cut -f2,7-10'),
    sep='\t', data.table=F
)

In [21]:
ref_alt_df <- data.frame(
    rsID = c('rs200058074', 'rs28991002', 'rs28991009', 'rs143435072'),
    ref  = c('A', 'G', 'G', 'C'),
    alt  = c('G', 'A', 'T', 'T')
) %>%
mutate(
    rsID_ref = paste0(rsID, '_', ref),
    raw_2 = paste0(ref, '/', ref),
    raw_1 = paste0(ref, '/', alt),
    raw_0 = paste0(alt, '/', alt)
)

In [82]:
geno_count <- raw_df %>% 
gather(
    rsID_ref, genotype, -IID
) %>% 
inner_join(
    ref_alt_df %>% select(-ref, -alt), by='rsID_ref'
) %>% 
mutate(
    genotype_str = genotype,
     genotype_str = if_else(genotype == 2, raw_2, 
                            if_else(genotype == 1, raw_1, 
                                    if_else(genotype == 0, raw_0, 'NA'))),
) %>% select(-raw_2, -raw_1, -raw_0, -rsID_ref, -genotype) %>% 
spread(rsID, genotype_str) %>% 
count(
    rs200058074, rs28991002, rs28991009, rs143435072
) %>% arrange(-n)

In [143]:
geno_count

rs200058074,rs28991002,rs28991009,rs143435072,n
A/A,G/G,G/G,C/C,328492
A/A,G/G,G/T,C/C,5395
A/A,G/A,G/G,C/C,1693
A/A,G/G,NA,C/C,471
A/G,G/G,G/G,C/C,356
A/A,G/G,G/G,C/T,270
A/A,NA,G/G,C/C,194
NA,G/G,G/G,C/C,192
A/A,G/G,G/G,NA,37
A/A,G/G,T/T,C/C,28


In [150]:
geno_count_expected <- geno_count %>% select(-n) %>%
mutate(combination = dplyr::row_number()) %>%
gather(
    variant, genotype, -combination
) %>% 
left_join(
    expected_genotype_freq %>%
    mutate(log10_expected = -log10(expected)) %>%
    rename(variant = ID) %>%
    select(-expected),
    by=c('variant', 'genotype')
) %>% 
replace_na(list(log10_expected = 0)) %>% 
group_by(combination) %>%
summarise(
    expected_freq = 10 ** (-sum(log10_expected))
) %>%
inner_join(
    geno_count %>% 
    mutate(combination = dplyr::row_number()),
    by='combination'
) %>%
mutate(
    n_expected = expected_freq * cohort_size
) %>%
select(-expected_freq, -combination)

In [153]:
geno_count_expected %>% fwrite(
    'ANGPTL7.protein-altering.vars.counts.tsv', sep='\t', row.names=FALSE, na = "NA"
)